In [10]:
import json
import numpy as np

def linear(x):
    return x

def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

def forward_propagation(model, input_data, weights):
    layers = model["layers"]
    output_layer = [input_data]

    for i, layer in enumerate(layers):
        activation_function = layer["activation_function"]
        weight_matrix = weights[i]
        bias = layer.get("bias", 1)

        input_activation = output_layer[-1]
        # print("Dimensi input_activation:", input_activation.shape)
        # print(input_activation)
        if len(input_activation.shape) == 1:
            input_activation_with_bias = np.insert(input_activation, 0, bias)
            input_activation_with_bias = input_activation_with_bias.reshape(1, -1)
        else:
            batch_size = input_activation.shape[0]
            bias_vector = np.ones((batch_size, 1)) * bias
            input_activation_with_bias = np.concatenate((bias_vector, input_activation), axis=1)
        # print("Dimensi input_activation_with_bias:", input_activation_with_bias.shape)
        # print(input_activation_with_bias)
        # print("Dimensi weight_matrix:", weight_matrix.shape)
        # print(weight_matrix)
        
        output_linear_combination = np.dot(input_activation_with_bias, weight_matrix)

        if activation_function == "linear":
            activation_result = linear(output_linear_combination)
        elif activation_function == "relu":
            activation_result = relu(output_linear_combination)
        elif activation_function == "sigmoid":
            activation_result = sigmoid(output_linear_combination)
        elif activation_function == "softmax":
            activation_result = softmax(output_linear_combination)

        output_layer.append(activation_result)

    return output_layer[-1]

In [11]:
# Load JSON input
with open("test/relu.json", "r") as file:
    data = json.load(file)

# Extract data
model = data["case"]["model"]
input_data = np.array(data["case"]["input"])
weights = [np.array(layer_weights) for layer_weights in data["case"]["weights"]]
expected_output = np.array(data["expect"]["output"])

In [12]:
# Perform forward propagation
output = forward_propagation(model, input_data, weights)

# Check error
error = np.sum((output - expected_output) ** 2)

# Check if error is within tolerance
max_sse = data["expect"]["max_sse"]
if error <= max_sse:
    print("Output matches the expected output within tolerance. With error of", error, "which is less than or equal to", max_sse, ".")
else:
    print("Output does not match the expected output within tolerance. With error of", error, "which is greater than", max_sse, ".")

print("Output:")
print(output)

Output matches the expected output within tolerance. With error of 4.8148248609680896e-33 which is less than or equal to 1e-06 .
Output:
[[0.05 1.1  0.  ]]
